Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE: MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework: MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and limitations under the License.

MindSPONGE tutorial protein 03: Sampling Process: Minimization - NVT - NPT - Product.

In [1]:
import os
os.environ['GLOG_v'] = '4'
os.environ['MS_JIT_MODULES'] = 'sponge'
from mindspore import context
import sys
sys.path.append('../../../src')
from sponge import ForceField
from sponge.system import get_molecule
from sponge import set_global_units, WithEnergyCell
from sponge.core import Sponge
from sponge.callback import WriteH5MD, RunInfo
from sponge.function import VelocityGenerator
from sponge import UpdaterMD
from mindspore import Tensor

[WARNING] ME(19107:140642485343424,MainProcess):2023-08-15-16:19:47.297.388 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(19107:140642485343424,MainProcess):2023-08-15-16:19:47.299.037 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(19107:140642485343424,MainProcess):2023-08-15-16:19:47.299.924 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(19107:140642485343424,MainProcess):2023-08-15-16:19:47.300.743 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(19107:140642485343424,MainProcess):2023-08-15-16:19:47.301.558 [mindspore/common/api.py:897] 'mindspore.ms_function

In [2]:
context.set_context(mode=context.GRAPH_MODE)
set_global_units('nm', 'kj/mol')

use the conformation from tutorial_p02

In [3]:
mol = get_molecule('./p02.pdb', template=['protein0.yaml', 'water.spce.yaml'])
mol.set_pbc_box(pbc_box=Tensor([2.1184928, 2.285693, 1.549593]))
energy = ForceField(mol, parameters=['AMBER.FF14SB', 'SPCE'], use_pme=False)
sim = WithEnergyCell(mol, energy)

NVT(control the temperature)

In [4]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(mol.shape, mol.atom_mass)
opt = UpdaterMD(system=mol,
                time_step=1e-3,
                velocity=velocity,
                integrator='velocity_verlet',
                temperature=300,
                thermostat='langevin',)
md = Sponge(sim, optimizer=opt)
cb_h5md = WriteH5MD(mol, './p03_1.h5md', save_freq=10, write_image=False)
run_info = RunInfo(10)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:19:56
[MindSPONGE] Step: 0, E_pot: 6321.4424, E_kin: 2124.98, E_tot: 8446.422, Temperature: 297.87454, Pressure: -1464.2379, Volume: 7.5034766
[MindSPONGE] Step: 10, E_pot: 7006.0483, E_kin: 1462.6494, E_tot: 8468.697, Temperature: 205.03064, Pressure: 432.76035, Volume: 7.5034766
[MindSPONGE] Step: 20, E_pot: 7105.43, E_kin: 1332.7485, E_tot: 8438.179, Temperature: 186.82146, Pressure: 54.21252, Volume: 7.5034766
[MindSPONGE] Step: 30, E_pot: 7306.297, E_kin: 1210.5671, E_tot: 8516.864, Temperature: 169.69437, Pressure: -404.85294, Volume: 7.5034766
[MindSPONGE] Step: 40, E_pot: 7413.882, E_kin: 1177.2192, E_tot: 8591.102, Temperature: 165.01974, Pressure: -1041.9178, Volume: 7.5034766
[MindSPONGE] Step: 50, E_pot: 7516.3125, E_kin: 1137.4458, E_tot: 8653.758, Temperature: 159.4444, Pressure: -1041.3145, Volume: 7.5034766
[MindSPONGE] Step: 60, E_pot: 7594.9727, E_kin: 1116.8837, E_tot: 8711.856, Temperature: 156.56204, Pressure: -950.8

NPT(control the pressure)

In [5]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(mol.shape, mol.atom_mass)
opt = UpdaterMD(system=mol,
                time_step=1e-3,
                velocity=velocity,
                integrator='velocity_verlet',
                temperature=300,
                thermostat='langevin',
                pressure=1,
                barostat='berendsen',)
md.change_optimizer(opt)
cb_h5md = WriteH5MD(mol, './p03_2.h5md', save_freq=10, write_image=False)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:20:28
[MindSPONGE] Step: 0, E_pot: 8104.9707, E_kin: 2187.0476, E_tot: 10292.019, Temperature: 306.575, Pressure: -1749.7047, Volume: 7.5034766
[MindSPONGE] Step: 10, E_pot: 8242.478, E_kin: 2074.7004, E_tot: 10317.178, Temperature: 290.82648, Pressure: -375.12863, Volume: 7.4936457
[MindSPONGE] Step: 20, E_pot: 8264.718, E_kin: 2031.6707, E_tot: 10296.389, Temperature: 284.79465, Pressure: -390.0842, Volume: 7.4864154
[MindSPONGE] Step: 30, E_pot: 8242.713, E_kin: 2054.4924, E_tot: 10297.205, Temperature: 287.99377, Pressure: -1337.9696, Volume: 7.4759717
[MindSPONGE] Step: 40, E_pot: 8311.836, E_kin: 1991.3667, E_tot: 10303.203, Temperature: 279.14496, Pressure: -687.186, Volume: 7.4663477
[MindSPONGE] Step: 50, E_pot: 8280.857, E_kin: 2042.9137, E_tot: 10323.771, Temperature: 286.3707, Pressure: -1399.444, Volume: 7.4591537
[MindSPONGE] Step: 60, E_pot: 8324.696, E_kin: 2021.7812, E_tot: 10346.478, Temperature: 283.4084, Pressure: -23

Product simulation(10000 steps for example)

In [6]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(mol.shape, mol.atom_mass)
opt = UpdaterMD(system=mol,
                time_step=1e-3,
                velocity=velocity,
                integrator='velocity_verlet',
                temperature=300,
                thermostat='langevin',
                pressure=1,
                barostat='berendsen',)
md.change_optimizer(opt)
cb_h5md = WriteH5MD(mol, './p03_3.h5md', save_freq=10, write_image=False)
md.run(10000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:21:02
[MindSPONGE] Step: 0, E_pot: 8096.7686, E_kin: 2079.098, E_tot: 10175.866, Temperature: 291.4429, Pressure: -1394.9108, Volume: 6.948632
[MindSPONGE] Step: 10, E_pot: 8069.3115, E_kin: 2128.435, E_tot: 10197.746, Temperature: 298.35886, Pressure: -2348.8018, Volume: 6.9428296
[MindSPONGE] Step: 20, E_pot: 8129.0967, E_kin: 2049.5623, E_tot: 10178.659, Temperature: 287.30267, Pressure: -1800.3663, Volume: 6.9373236
[MindSPONGE] Step: 30, E_pot: 8036.3086, E_kin: 2101.9214, E_tot: 10138.23, Temperature: 294.64224, Pressure: -1871.82, Volume: 6.9321246
[MindSPONGE] Step: 40, E_pot: 8026.169, E_kin: 2091.064, E_tot: 10117.232, Temperature: 293.1203, Pressure: -441.88065, Volume: 6.92916
[MindSPONGE] Step: 50, E_pot: 7978.0635, E_kin: 2123.5784, E_tot: 10101.642, Temperature: 297.67807, Pressure: -1633.9957, Volume: 6.926644
[MindSPONGE] Step: 60, E_pot: 7918.588, E_kin: 2172.5554, E_tot: 10091.144, Temperature: 304.54355, Pressure: -11